<a href="https://colab.research.google.com/github/kamal1262/scrape_plane_crush_dataset/blob/master/multiclass_classification_based_on_airline_crashes_summary_info.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

!pip uninstall tensorflow
!pip install tensorflow-gpu==2.0.0
!pip install ktrain 

Uninstalling tensorflow-1.15.0:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/freeze_graph
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.6/dist-packages/tensorflow-1.15.0.dist-info/*
    /usr/local/lib/python3.6/dist-packages/tensorflow/*
    /usr/local/lib/python3.6/dist-packages/tensorflow_core/*
Proceed (y/n)? y
  Successfully uninstalled tensorflow-1.15.0
     |████████████████████████████████| 380.8MB 46kB/s 
     |████████████████████████████████| 450kB 41.9MB/s 
     |████████████████████████████████| 3.8MB 48.7MB/s 
     |████████████████████████████████| 81kB 13.3MB/s 
ERROR: tensorboard 2.0.2 has requirement grpcio>=1.24.3, but you'll have grpcio 1.15.0 which is incompatible.
ERROR: google-colab 1.0.0 has requirement google-auth~=1.4.0, but you'll have google-auth 1.11.0 w

     |████████████████████████████████| 102kB 3.3MB/s 
     |████████████████████████████████| 6.7MB 9.4MB/s 
     |████████████████████████████████| 1.0MB 53.0MB/s 
     |████████████████████████████████| 245kB 58.6MB/s 
     |████████████████████████████████| 1.8MB 53.2MB/s 
     |████████████████████████████████| 481kB 58.2MB/s 
     |████████████████████████████████| 3.1MB 45.3MB/s 
     |████████████████████████████████| 870kB 62.5MB/s 
     |████████████████████████████████| 1.0MB 56.6MB/s 
  Created wheel for ktrain: filename=ktrain-0.9.0-cp36-none-any.whl size=123690 sha256=0f0e2251196e5e675181695eb1da1658080fde391e74d4472e34cf8acad3b945
  Stored in directory: /root/.cache/pip/wheels/c7/02/23/a1a3fa435b278f3a1e650b405db286451cdff0ddeee9bb9ba1
  Created wheel for keras-bert: filename=keras_bert-0.81.0-cp36-none-any.whl size=37913 sha256=43d4315473acb2292cc7cf02024c457be93a0bada882ab85d28aada9f2873375
  Stored in directory: /root/.cache/pip/wheels/bd/27/da/ffc2d573aa48b87440ec4f9

In [ ]:
# import tensorflow as tf 
# print(tf.__version___)

import tensorflow as tf

tf.__version__

'2.0.0'

In [ ]:

%reload_ext autoreload
%autoreload 2
%matplotlib inline
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
os.environ["CUDA_VISIBLE_DEVICES"]="0";

categories = ['alt.atheism', 'soc.religion.christian',
             'comp.graphics', 'sci.med']
from sklearn.datasets import fetch_20newsgroups
train_b = fetch_20newsgroups(subset='train',
   categories=categories, shuffle=True, random_state=42)
test_b = fetch_20newsgroups(subset='test',
   categories=categories, shuffle=True, random_state=42)

print('size of training set: %s' % (len(train_b['data'])))
print('size of validation set: %s' % (len(test_b['data'])))
print('classes: %s' % (train_b.target_names))

x_train = train_b.data
y_train = train_b.target
x_test = test_b.data
y_test = test_b.target

size of training set: 2257
size of validation set: 1502
classes: ['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']


In [ ]:
set(y_train)

{0, 1, 2, 3}

In [ ]:
import ktrain
from ktrain import text

(x_train,  y_train), (x_test, y_test), preproc = text.texts_from_array(x_train=x_train, y_train=y_train,
                                                                       x_test=x_test, y_test=y_test,
                                                                       class_names=train_b.target_names,
                                                                       preprocess_mode='bert',
                                                                       ngram_range=1, 
                                                                       maxlen=350, 
                                                                       max_features=35000)

using Keras version: 2.2.4-tf
downloading pretrained BERT model (uncased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: en


preprocessing test...
language: en


In [ ]:
model = text.text_classifier('bert', train_data=(x_train, y_train), preproc=preproc)


Is Multi-Label? False
maxlen is 350
done.


In [ ]:
learner = ktrain.get_learner(model, train_data=(x_train, y_train), batch_size=6)


In [ ]:
learner.autofit(2e-5, 5)



begin training using triangular learning rate policy with max lr of 2e-05...
Train on 2257 samples
Epoch 1/5
2257/2257 [==============================] - 301s 133ms/sample - loss: 0.4816 - accuracy: 0.8148
Epoch 2/5
2257/2257 [==============================] - 282s 125ms/sample - loss: 0.0858 - accuracy: 0.9739
Epoch 3/5
2257/2257 [==============================] - 281s 125ms/sample - loss: 0.0546 - accuracy: 0.9827
Epoch 4/5
  30/2257 [..............................] - ETA: 4:39 - loss: 0.0080 - accuracy: 1.0000

In [ ]:
learner.validate(val_data=(x_test, y_test))

In [ ]:

predictor = ktrain.get_predictor(learner.model, preproc)
predictor.get_classes()

In [ ]:
print(predictor.predict(test_b.data[0:3]))
print(test_b.target[:3])

### **DistillBERT**

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
os.environ["CUDA_VISIBLE_DEVICES"]="0";

In [ ]:

import ktrain
from ktrain import text

using Keras version: 2.2.4-tf


## Load the Data Into **Arrays**

In [ ]:
categories = ['alt.atheism', 'soc.religion.christian',
             'comp.graphics', 'sci.med']
from sklearn.datasets import fetch_20newsgroups
train_b = fetch_20newsgroups(subset='train',
   categories=categories, shuffle=True, random_state=42)
test_b = fetch_20newsgroups(subset='test',
   categories=categories, shuffle=True, random_state=42)

print('size of training set: %s' % (len(train_b['data'])))
print('size of validation set: %s' % (len(test_b['data'])))
print('classes: %s' % (train_b.target_names))

x_train = train_b.data
y_train = train_b.target
x_test = test_b.data
y_test = test_b.target

size of training set: 2257
size of validation set: 1502
classes: ['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']


# ***STEP 1: Preprocess Data***

In [ ]:
trn, val, preproc = text.texts_from_array(x_train=x_train, y_train=y_train,
                                          x_test=x_test, y_test=y_test,
                                          class_names=train_b.target_names,
                                          preprocess_mode='distilbert',
                                          maxlen=350)

preprocessing train...
language: en


preprocessing test...
language: en


# STEP 2: Build a Model and Wrap in Learner

In [ ]:
model = text.text_classifier('distilbert', train_data=trn, preproc=preproc)
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=6)

Is Multi-Label? False
maxlen is 350
done.


# STEP 3: Train Model

In [ ]:
learner.fit_onecycle(3e-5, 4)



begin training using onecycle policy with max lr of 3e-05...
Train for 377 steps, validate for 251 steps
Epoch 1/4
377/377 [==============================] - 89s 235ms/step - loss: 0.6645 - accuracy: 0.8042 - val_loss: 0.2394 - val_accuracy: 0.9348
Epoch 2/4
377/377 [==============================] - 75s 199ms/step - loss: 0.1175 - accuracy: 0.9712 - val_loss: 0.4541 - val_accuracy: 0.8296
Epoch 3/4
377/377 [==============================] - 75s 199ms/step - loss: 0.0719 - accuracy: 0.9787 - val_loss: 0.1624 - val_accuracy: 0.9561
Epoch 4/4
377/377 [==============================] - 75s 198ms/step - loss: 0.0140 - accuracy: 0.9982 - val_loss: 0.1582 - val_accuracy: 0.9547


In [ ]:
x_train[0]

'From: sd345@city.ac.uk (Michael Collier)\nSubject: Converting images to HP LaserJet III?\nNntp-Posting-Host: hampton\nOrganization: The City University\nLines: 14\n\nDoes anyone know of a good way (standard PC application/PD utility) to\nconvert tif/img/tga files into LaserJet III format.  We would also like to\ndo the same, converting to HPGL (HP plotter) files.\n\nPlease email any response.\n\nIs this the correct group?\n\nThanks in advance.  Michael.\n-- \nMichael Collier (Programmer)                 The Computer Unit,\nEmail: M.P.Collier@uk.ac.city                The City University,\nTel: 071 477-8000 x3769                      London,\nFax: 071 477-8565                            EC1V 0HB.\n'

In [ ]:
set(y_train)

{0, 1, 2, 3}

## Predict on New **Data**

In [ ]:
p = ktrain.get_predictor(model, preproc)
p.predict("There is a problem with my computer monitor's resolution.  Everything is blurry.")

'comp.graphics'

In [ ]:
p.predict(x_train[2])

'soc.religion.christian'

In [ ]:
x_train[2]

"From: djohnson@cs.ucsd.edu (Darin Johnson)\nSubject: Re: harrassed at work, could use some prayers\nOrganization: =CSE Dept., U.C. San Diego\nLines: 63\n\n(Well, I'll email also, but this may apply to other people, so\nI'll post also.)\n\n>I've been working at this company for eight years in various\n>engineering jobs.  I'm female.  Yesterday I counted and realized that\n>on seven different occasions I've been sexually harrassed at this\n>company.\n\n>I dreaded coming back to work today.  What if my boss comes in to ask\n>me some kind of question...\n\nYour boss should be the person bring these problems to.  If he/she\ndoes not seem to take any action, keep going up higher and higher.\nSexual harrassment does not need to be tolerated, and it can be an\nenormous emotional support to discuss this with someone and know that\nthey are trying to do something about it.  If you feel you can not\ndiscuss this with your boss, perhaps your company has a personnel\ndepartment that can work for y